# KODIAQ

In [18]:
# imports
import pandas
import numpy as np

from matplotlib import pyplot as plt

from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units

from specdb import specdb

# Load KOA

In [2]:
all_qsos = pandas.read_fwf('qso_results.txt')

In [3]:
all_qsos['total'] = all_qsos['nires'] + all_qsos['mosfire'] + all_qsos['nirspec'] + all_qsos['esi'] + all_qsos['deimos'] + all_qsos['hires']

In [10]:
qsos = Table.read('qso_rsg4p5_v2.0_all_20200609.fits')

# Load KODIAQ

In [5]:
igmspec = specdb.IgmSpec()

Loading igmspec from /data/spec_DB/IGMspec_DB_v03.1.hdf5


/data/Projects/Python/specdb/specdb/query_catalog.py:69: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.cat = Table(hdf['catalog'].value)


Database is igmspec
Created on 2019-Mar-06
Version: v03.1


In [6]:
igmspec.groups

['BOSS_DR14',
 'SQUAD_DR1',
 'ESI_z6',
 'KODIAQ_DR2',
 'SDSS_DR7',
 'KODIAQ_DR1',
 'HD-LLS_DR1',
 'GGG',
 'HST_z2',
 'XQ-100',
 'HDLA100',
 '2QZ',
 'ESI_DLA',
 'COS-Halos',
 'COS-Dwarfs',
 'HSTQSO',
 'MUSoDLA',
 'UVES_Dall',
 'UVpSM4']

# Coords

## KOA

In [16]:
subset = all_qsos['total'] > 0
sub_koa = qsos[subset.values]
len(sub_koa)

194

In [19]:
koa_coord = SkyCoord(ra=sub_koa['RA'], dec=sub_koa['DEC'], frame='icrs', unit=(units.hourangle, units.deg))

## KODIAQ1

In [24]:
kodiaq1 = igmspec['KODIAQ_DR1'].meta
kodiaq1[0:3]

zem_GROUP,kodetime,kodwblue,kodwred,kodrelease,RA_GROUP,DEC_GROUP,sig_zem,EPOCH,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,IGM_ID,qso,sRA,sDEC,pi_date,spec_prefix,redux_setup,targname,deckname,elaptime,qaflag,DATE-OBS,STYPE,flag_zem,SPEC_FILE,INSTR,TELESCOPE,DISPERSER
float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,str14,str11,str12,str27,str20,str8,str14,str2,str18,str4,str10,str3,str6,str27,str5,str6,str4
2.350,1200,3334,6198,1,2.38096,2.28542,0.0,2000.0,71497,3334.1,6198.2,48000.0,0,526970,J000931+021707,00:09:31.43,+02:17:07.5,Tytler_Dec_10_12_13_14_2009,J000931+021707,Setup_2,0009+0217,C1,1200,ok,2009-12-10,QSO,SIMBAD,J000931+021707_f.fits,HIRES,Keck-I,BLUE
2.596,700,3366,6198,1,5.36396,-2.05933,0.0,2000.0,70404,3365.8,6198.2,48000.0,1,526971,J002127-020333,00:21:27.35,-02:03:33.6,Tytler_Dec_10_12_13_14_2009,J002127-020333,Setup_2,0021-0203,C1,700,ok,2009-12-10,QSO,SIMBAD,J002127-020333_f.fits,HIRES,Keck-I,BLUE
2.400,600,3366,6198,1,7.12683,-28.28467,0.0,2000.0,70404,3365.9,6198.2,48000.0,2,526972,J002830-281704,00:28:30.44,-28:17:04.8,Tytler_Dec_10_12_13_14_2009,J002830-281704,Setup_2,0028-2817,C1,600,ok,2009-12-10,QSO,SIMBAD,J002830-281704_f.fits,HIRES,Keck-I,BLUE


In [25]:
kodiaq1_coord = SkyCoord(ra=kodiaq1['RA_GROUP'], dec=kodiaq1['DEC_GROUP'], unit='deg')

## KODIAQ2

In [26]:
kodiaq2 = igmspec['KODIAQ_DR2'].meta
kodiaq2[0:3]

/data/Projects/Python/specdb/specdb/interface_group.py:70: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.meta = spdbu.hdf_decode(self.hdf[group+'/meta'].value, itype='Table')


zem_GROUP,kodetime,kodwblue,kodwred,kodrelease,RA_GROUP,DEC_GROUP,sig_zem,IGM_ID,EPOCH,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,qso,sRA,sDEC,pi_date,spec_prefix,redux_setup,targname,deckname,elaptime,qaflag,DATE-OBS,INSTR,TELESCOPE,STYPE,flag_zem,SPEC_FILE,DISPERSER
float64,int64,int64,int64,int64,float64,float64,float64,int64,float64,int64,float64,float64,float64,int64,str14,str11,str12,str31,str20,str8,str14,str2,str18,str4,str10,str5,str6,str3,str11,str27,str4
2.810,25000,4111,6520,2,0.45833,-1.99444,0.0,1088,2000.0,53189,4110.7,6520.0,36000.0,0,J000150-015940,00:01:50.0,-01:59:40,Wolfe_Sep_29_30_1997,J000150-015940,Setup_3,Q2359-OBJ,C5,"2x7200,2x5400",ok,1997-09-29,HIRES,Keck-I,QSO,SDSS-SIMBAD,J000150-015940_f.fits,RED
1.887,8000,3307,4849,2,1.33421,5.40300,0.0,447263,2000.0,44149,3306.7,4849.3,48000.0,1,J000520+052410,00:05:20.21,+05:24:10.79,Sargent_Oct_12_13_14_1999,J000520+052410,Setup_1,Q0002+0507,C1,2x4000,ok,1999-10-12,HIRES,Keck-I,QSO,SDSS-SIMBAD,J000520+052410_f.fits,BLUE
1.887,8000,3340,4888,2,1.33421,5.40300,0.0,447263,2000.0,43918,3339.9,4888.2,48000.0,2,J000520+052410,00:05:20.21,+05:24:10.79,Sargent_Oct_12_13_14_1999,J000520+052410_spec2,Setup_2,Q0002+0507,C1,2x4000,ok,1999-10-12,HIRES,Keck-I,QSO,SDSS-SIMBAD,J000520+052410_spec2_f.fits,BLUE


In [27]:
kodiaq2_coord = SkyCoord(ra=kodiaq2['RA_GROUP'], dec=kodiaq2['DEC_GROUP'], unit='deg')

## KODIAQ3

In [31]:
kodiaq3 = Table.read('DR3_summary.txt', format='ascii')

In [32]:
kodiaq3[0:3]

Index,Name,RA,DEC,Ref,z,wvmin,wvmax,flag,s2n,s2nB
int64,str14,str13,str13,str19,float64,float64,float64,float64,float64,float64
1,J000211-052908,00:02:11.7696,-05:29:08.484,Martin_20111025,2.8,4000.0,10130.6,-1.0,24.741,34.043
2,J000216-053007,00:02:16.668,-05:30:07.632,Martin_20111025,3.1592,4000.0,10130.6,-1.0,25.504,31.442
3,J000221+002149,00:02:21.1176,+00:21:49.356,Becker_20001229,3.05703,4000.0,10134.9,-1.0,14.029,18.327


In [33]:
kodiaq3_coord = SkyCoord(ra=kodiaq3['RA'], dec=kodiaq3['DEC'], frame='icrs', unit=(units.hourangle, units.deg))

# Match me!

In [35]:
for kk, kodiaq_coord in enumerate([kodiaq1_coord, kodiaq2_coord, kodiaq3_coord]):
    # Match
    idx, d2d, _ = match_coordinates_sky(koa_coord, kodiaq_coord, nthneighbor=1)
    # Close
    match = d2d < 2*units.arcsec
    # Answer
    print("KODIAQ{} has {} matches".format(kk+1, np.sum(match)))

KODIAQ1 has 4 matches
KODIAQ2 has 3 matches
KODIAQ3 has 108 matches


## Explore KODIAQ3

In [36]:
idx3, d2d3, _ = match_coordinates_sky(koa_coord, kodiaq3_coord, nthneighbor=1)

In [37]:
match3 = d2d3 < 2 * units.arcsec

In [42]:
kodiaq3[idx3[match3]]['z'].data

array([6.438   , 6.4189  , 6.427   , 6.311   , 6.247   , 6.25    ,
       6.2284  , 6.13    , 6.13    , 6.118   , 6.12    , 6.117   ,
       6.07    , 6.065   , 6.049   , 6.04    , 6.0379  , 6.016   ,
       6.01    , 6.02    , 6.13    , 5.98    , 5.9012  , 5.927   ,
       5.85    , 5.82    , 5.8441  , 5.7824  , 5.7722  , 5.72    ,
       5.62    , 5.41353 , 5.3383  , 5.33    , 5.31312 , 5.305602,
       5.31775 , 5.290043, 5.273   , 5.2924  , 5.20557 , 5.167925,
       5.214469, 5.1877  , 5.14174 , 5.16359 , 5.13512 , 5.27264 ,
       5.092134, 5.10933 , 5.068   , 5.1195  , 5.10693 , 5.0589  ,
       5.07091 , 5.051909, 4.99    , 5.02249 , 5.005901, 4.976896,
       4.95    , 4.92177 , 4.9     , 4.92302 , 4.918688, 4.908   ,
       4.91233 , 4.897   , 4.996   , 4.897885, 4.881   , 4.88142 ,
       4.85927 , 4.79034 , 4.935   , 4.87299 , 4.88504 , 4.81    ,
       4.904886, 4.78947 , 4.846415, 4.795   , 4.784268, 4.772255,
       4.79807 , 4.754505, 4.70677 , 4.70859 , 4.707273, 4.774

In [43]:
m3_z4 = kodiaq3[idx3[match3]]['z'].data > 4
np.sum(m3_z4)

108